In [2]:
%load_ext aiida
%aiida
from IPython.display import display, clear_output, Markdown
import ipywidgets as ipw
import widgets
import utils
import aiida_utils
import json
import shutil

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
slabs_options = [DATA_MODEL["classes"][object_type]["title"] for object_type in CONFIG["slabs_concepts"]]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

# material_selector_config = {
#     "dropdown": {"width": "315px"},
#     "details": {"width": "415px", "height": "250px"},
#     "image": {"width": "220px", "height": "250px"}
# }
# material_selector = widgets.ObjectSelectionWidget("Material", material_selector_config)
material_selector = widgets.MaterialSelectionWidget()

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

molecule_selector_list = widgets.MultipleSelectorWidget("molecule_concept")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule concept', disabled = False, button_style = '', 
    tooltip = 'Add molecule concept', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule concept', disabled = False, button_style = '', 
    tooltip = 'Remove molecule concept', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

product_selector_list = widgets.MultipleSelectorWidget("product_concept")
product_selector_list_output = ipw.Output()

add_product_button = utils.Button(
    description = 'Add product concept', disabled = False, button_style = '', 
    tooltip = 'Add product concept', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_product_button = utils.Button(
    description = 'Remove product', disabled = False, button_style = '', 
    tooltip = 'Remove product', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_product_buttons_hbox = ipw.HBox([add_product_button, remove_product_button])

is_automatic_checkbox = utils.Checkbox(
    description = 'Simulation developed using AiiDA', value = False, 
    disabled = False, layout = ipw.Layout(width = "250px"), indent = False
)

simulation_details_output = ipw.Output()

simulation_type_dropdown = utils.Dropdown(
    description = "Simulation type", disabled = False, 
    layout = ipw.Layout(width = "300px"), 
    options = [("Select an option...", -1), 
               ("Geometry Optimisation", "GeometryOptimisation"), 
               ("Band Structure", "BandStructure"), 
               ("PDOS", "PDOS"), 
               ("Vibrational Spectroscopy", "VibrationalSpectroscopy"), 
               ("Unclassified Simulation", "UnclassifiedSimulation")],
    value = -1,
    style = {'description_width': "110px"}    
)

atomistic_model_selector_config = {
    "dropdown": {"width": "315px"},
    "details": {"width": "415px", "height": "250px"},
    "image": {"width": "220px", "height": "250px"}
}
atomistic_model_selector = widgets.ObjectSelectionWidget("AtomisticModel", atomistic_model_selector_config)
atomistic_model_selector.load_dropdown_box()

simulation_properties_widgets = {}
simulation_properties_output = ipw.Output()
for simulation_type in simulation_type_dropdown.options[1:]:
    simulation_type = simulation_type[1]
    simulation_widget = widgets.ObjectPropertiesWidgets(simulation_type)
    simulation_widget.get_properties_widgets()
    simulation_properties_widgets[simulation_type] = simulation_widget

simulation_selector = widgets.ObjectSelectionWidget("Simulation", checkboxes_descriptions = ["Name", "PK"])
simulation_selector.load_dropdown_box(source = "aiidalab")

code_selector = widgets.ObjectMultipleSelectionWidget("Codes")
code_selector.load_selector("CODE")

simulation_dataset_upload = ipw.FileUpload(multiple = True)
simulation_preview_upload = ipw.FileUpload(multiple = False, accept = '.png, .jpg, .jpeg')
simulation_images_upload = ipw.FileUpload(multiple = True, accept = '.png, .jpg, .jpeg, .tar')
simulation_support_files_upload = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def select_material_radio_change(change):
    material_title = material_selection_radio_button.value
    if material_title == "No material":
        with material_selector:
            clear_output()
            material_selector.dropdown.value = -1
            return
    
    material_selector.details_textbox.value = ''
    for object_type in CONFIG["slabs_concepts"]:
        object_schema = DATA_MODEL["classes"][object_type]
        object_title = object_schema["title"]
        if material_title == object_title:
            material_type = object_type
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_selector.load_dropdown_box(material_type)
        display(ipw.VBox(display_list))

def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def add_product_widget(b):
    product_selector_list.add_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def remove_product_widget(b):
    product_selector_list.remove_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)
      
def create_simulation_openbis(b):
    if experiment_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select an experiment.')"))
        return
    else:
        object_parents = []
        if molecule_selector_list.selectors:
            for molecule_selector in molecule_selector_list.selectors:
                if molecule_selector.dropdown.value != -1:
                    object_parents.append(molecule_selector.dropdown.value)
        if material_selector.dropdown.value != -1:
            object_parents.append(material_selector.dropdown.value)
        if product_selector_list.selectors:
            for product_selector in product_selector_list.selectors:
                if product_selector.dropdown.value != -1:
                    object_parents.append(product_selector.dropdown.value)
                    
        if is_automatic_checkbox.value:
            last_export = aiida_utils.export_workchain(
                OPENBIS_SESSION, 
                experiment_selector.dropdown.value, 
                simulation_selector.dropdown.value
            )
            
            if last_export:
                first_atomistic_model = utils.find_first_atomistic_model(OPENBIS_SESSION, last_export, "ATOMISTIC_MODEL")
                if len(first_atomistic_model.parents) == 0:
                    first_atomistic_model.parents = object_parents
                    first_atomistic_model.save()
                display(utils.Javascript(data = "alert('Upload successful!')"))
            else:
                display(utils.Javascript(data = "alert('Simulation is already in openBIS!')"))
        else:
            if simulation_type_dropdown.value == "Select an option...":
                display(utils.Javascript(data = "alert('Select a simulation type.')"))
                return
            else:
                simulation_type = simulation_type_dropdown.value
                simulation_openbis_type = CONFIG["objects"][simulation_type]["openbis_object_type"]
                simulation_widgets = simulation_properties_widgets[simulation_type_dropdown.value]
                object_properties = utils.get_widget_values(simulation_widgets.properties_widgets_detailed_dict)
                
                if atomistic_model_selector.dropdown.value != -1:
                    object_parents.append(atomistic_model_selector.dropdown.value)

                object_parents.extend(code_selector.selector.value)
                
                simulation_object = utils.create_openbis_object(
                    OPENBIS_SESSION, type=simulation_openbis_type, 
                    collection = experiment_selector.dropdown.value, 
                    props = object_properties,
                    parents = object_parents
                )

                utils.upload_datasets(OPENBIS_SESSION, simulation_object, simulation_dataset_upload, "RAW_DATA")
                utils.upload_datasets(OPENBIS_SESSION, simulation_object, simulation_preview_upload, "ELN_PREVIEW")
                utils.upload_datasets(OPENBIS_SESSION, simulation_object, simulation_images_upload, "RAW_DATA")
                utils.upload_datasets(OPENBIS_SESSION, simulation_object, simulation_support_files_upload, "RAW_DATA")
                display(utils.Javascript(data = "alert('Upload successful!')"))
                simulation_properties_widgets[simulation_type].reset_properties_widgets()

def enter_simulation_details(change):
    with simulation_details_output:
        clear_output()
        display(Markdown("### Select molecules"))
        display(molecule_selector_list_output)
        with molecule_selector_list_output:
            display(molecule_selector_list)  
        display(add_remove_molecule_buttons_hbox)
        display(Markdown("### Select reaction products"))
        display(product_selector_list_output)
        with product_selector_list_output:
            display(product_selector_list)  
        display(add_remove_product_buttons_hbox)
        display(Markdown("### Select slab"))
        display(material_selection_radio_button)
        display(material_selector)
        
        if is_automatic_checkbox.value:
            display(Markdown("### Select simulation"))
            display(simulation_selector)
        else:
            display(Markdown("### Select simulation type"))
            display(simulation_type_dropdown)
            display(simulation_properties_output)   
            display(Markdown("### Select atomistic model"))
            display(atomistic_model_selector)
            display(Markdown("### Select code"))
            display(code_selector)
            display(Markdown("### Upload dataset"))
            display(simulation_dataset_upload)
            display(Markdown("### Upload image preview"))
            display(simulation_preview_upload)
            display(Markdown("### Upload other images"))
            display(simulation_images_upload)
            display(Markdown("### Upload other support files"))
            display(simulation_support_files_upload)

def load_simulation_properties_widgets(change):
    with simulation_properties_output:
        clear_output()
        if simulation_type_dropdown.value != "Select an option...":
            display(Markdown("### Enter simulation properties"))
            display(simulation_properties_widgets[simulation_type_dropdown.value])

def load_structure_material(change):
    if simulation_selector.dropdown.value != -1:
        simulation_pk = simulation_selector.dropdown.value
        simulation_uuid = load_node(simulation_pk).uuid
        first_structure_uuid = aiida_utils.original_structure(simulation_uuid)
        first_structure_extras = load_node(first_structure_uuid).extras
        
        first_structure_permid = first_structure_extras["eln"]["sample_uuid"]
        first_structure_object = OPENBIS_SESSION.get_objects(first_structure_permid)
        
        if first_structure_object:
            if first_structure_extras["eln"]["data_type"] == "cdxml":
                add_product_widget(None)
                product_selector_list.selectors[0].dropdown.value = first_structure_permid
            else:
                add_molecule_widget(None)
                molecule_selector_list.selectors[0].dropdown.value = first_structure_permid

# Send simulation to openBIS

## Select experiment

In [ ]:
display(experiment_selector)

## Simulation details

In [ ]:
display(is_automatic_checkbox)
display(simulation_details_output)

## Save simulation workchain

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
is_automatic_checkbox.observe(enter_simulation_details, names = 'value')
enter_simulation_details(None)
atomistic_model_selector.dropdown.observe(atomistic_model_selector.load_metadata, names = 'value')
simulation_selector.dropdown.observe(load_structure_material, names = 'value')
simulation_type_dropdown.observe(load_simulation_properties_widgets, names = 'value')
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(material_selector.load_metadata, names = 'value')
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
add_product_button.on_click(add_product_widget)
remove_product_button.on_click(remove_product_widget)
create_button.on_click(create_simulation_openbis)
quit_button.on_click(close_notebook)